# Modeling
In this notebook, we'll be modeling the data we've previously prepared. Out notebook will be laid out as follows:

1. Model Selection & Generation
2. Hyperparameter Optimization
3. Fine-Tuning (if needed)
4. Reporting Best Model(s) + Settings
5. Interpretation
6. Conclusion

Our eventual goal here is two-fold:

1. Accurately [and fairly] model the diabetes dataset
2. Interpret the results to find something worth recommending to those wanting to reduce risk of diabetes. This can be via LIME/SHAP (i.e. some interpretable model that approximates the neural network) or via analyzing a more simple model's structure (i.e. regression coefficients, random forest decision boundaries)

In [1]:
# Environment Setup
from utils.model import *
from utils.dataset import *

***
## Model Selection & Generation

In [2]:
# generate lookup for models
models = {
    # "tree": TreeClassifier(target="diabetes", path="../datasets/processed.parquet"),
    "ffnn": MLPClassifier(target="diabetes", path="../datasets/processed.parquet")
}
models["ffnn"].set_hyperparams({"num_epochs": 1, "learning_rate": 0.001})

# train & test basic model
for mt, model in models.items():
    # attempt to load, else train and test
    if not model.load_model():
        model.train_model()
    model.test_model()

<Train-Test Split Report>
Train: 512887 obs, 170945 no diabetes [0], 171126 pre-diabetes [1], 170816 diabetes [2]
Test: 128222 obs, 42758 no diabetes [0], 42577 pre-diabetes [1], 42887 diabetes [2]
<WARNING> found 0 entries in model lookup :(


100%|██████████| 16028/16028 [00:20<00:00, 770.33it/s]


Epoch 1/1, Loss: 0.979275025101948

<Test Report>
Precision: [no diabetes] 0.6311167487925062, [pre-diabetes] 0.4506779752579547, [diabetes] 0.5154961558379476
Recall: [no diabetes] 0.6050797511576781, [pre-diabetes] 0.3918782441224135, [diabetes] 0.6034695828572761
F1-Score: [no diabetes] 0.6178240519629382, [pre-diabetes] 0.41922637219060543, [diabetes] 0.5560246205407495
Support: [no diabetes] 42758, [pre-diabetes] 42577, [diabetes] 42887
Accuracy: 53.3746%


***
## Hyperparameter Optimization

In [3]:
# optimize hyperparams
optimizer_results = {model_type: model.optimize_hyperparams(kfold=2) for model_type, model in models.items()}
print(optimizer_results)

InvalidParameterError: The 'estimator' parameter of GridSearchCV must be an object implementing 'fit'. Got LinearNN(
  (fc_input): Linear(in_features=21, out_features=64, bias=True)
  (hidden_layers): ModuleList(
    (0-1): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (fc_output): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
  (classify_fn): Sigmoid()
) instead.

***
## Fine-Tuning + Other Adjustments

***
## Best Model Report

***
## Interpretation

***
## Conclusion